In [ ]:
import imapclient
import smtplib

In [ ]:
imap_server = 'imap.gmail.com'
smtp_server = 'smtp.gmail.com'
username = 'mail_id'
password = 'password'

imapobj = imapclient.IMAPClient(imap_server, ssl=True)
imapobj.login(username, password)

smtpobj = smtplib.SMTP(smtp_server, 587)
smtpobj.ehlo()
smtpobj.starttls()
smtpobj.login(username, password)

In [ ]:

import imaplib
import pprint

pprint.pprint(imapobj.list_folders())

imaplib._MAXLINE = 10000000

imapobj.select_folder('Inbox', readonly=True)
UIDs = imapobj.search(['SINCE', '01-Aug-2022', 'BEFORE', '30-Sep-2022'])

In [ ]:
category = []
for i in range(len(UIDs)):
    label_dict = imapobj.get_gmail_labels(UIDs[i])
    label = label_dict[UIDs[i]]
    if 'Starred' in str(label):
        category.append('Starred')
    elif 'Important' in str(label):
        category.append('Important')
    elif len(label) == 0:
        category.append('Inbox')
    else:
        category.append('Custom Label')

In [ ]:
import pandas as pd
category1 = pd.DataFrame(category, columns=['Category'])
category1['Category'].value_counts()
category1.head()

In [ ]:
len(UIDs)

In [ ]:
import pyzmail
from_addresses = []
subjects = []
dates = []
days = []
months = []
years = []
times = []
sent_received = []
unsub_links = []

for i in range(len(UIDs)):
    raw_message = imapobj.fetch(UIDs[i], ['BODY[]'])
    message = pyzmail.PyzMessage.factory(raw_message[UIDs[i]][b'BODY[]'])

    if message.get_address('from')[1] == username:
        sent_received.append('Sent')
    else:
        sent_received.append('Received')

    full_date = message.get_decoded_header('date')
    from_addresses.append(message.get_address('from'))
    subjects.append(message.get_subject(''))

    day = full_date.split()[0].strip(',')
    date = full_date.split()[1]
    month = full_date.split()[2]
    year = full_date.split()[3]
    time = full_date.split()[4]

    days.append(day)
    dates.append(date)
    months.append(month)
    years.append(year)
    times.append(time)

In [ ]:
from openpyxl import Workbook
wb = Workbook()
ws = wb.active
ws.title = "email_info"
ws.cell(1,1).value = "Date"
ws.cell(1,2).value = "Month"
ws.cell(1,3).value = "Year"
ws.cell(1,4).value = "Day"
ws.cell(1,5).value = "Time"
ws.cell(1,6).value = "From (Sender)"
ws.cell(1,7).value = "From (Email ID)"
ws.cell(1,8).value = "Subject"
ws.cell(1,9).value = "Sent/Received"
ws.cell(1,10).value = "Category"
for i in range(len(UIDs)):
    ws.cell(row=i+2, column=1).value = dates[i]
    ws.cell(row=i+2, column=2).value = months[i]
    ws.cell(row=i+2, column=3).value = years[i]
    ws.cell(row=i+2, column=4).value = days[i]
    ws.cell(row=i+2, column=5).value = times[i]
    ws.cell(row = i+2, column = 6).value = from_addresses[i][0]
    ws.cell(row = i+2, column = 7).value = from_addresses[i][1]
    ws.cell(row = i+2, column = 8).value = str(subjects[i])
    ws.cell(row=i + 2, column=9).value = sent_received[i]
    ws.cell(row=i+2, column=10).value = category[i]

wb.save('Email_Analytics1.xlsx')

In [ ]:
import pandas as pd
data = pd.read_excel('Email_Analytics1.xlsx')


In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt
received = data[data['Sent/Received'] == 'Received']
sender_top_20 =  received['From (Sender)'].value_counts().nlargest(20)
sender_top_20_count = sender_top_20.values
sender_top_20_names = sender_top_20.index.tolist()

plt.figure()
plt.barh(sender_top_20_names, sender_top_20_count, color = 'forestgreen', ec = 'black', linewidth = 0.50)
plt.gca().invert_yaxis()
plt.title('Top 20 Senders', fontsize = 15 ,fontweight = 'bold')
plt.xlabel('Received Email Count', fontweight = 'bold')
plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
text = data['From (Sender)'].values
stopwords = set(STOPWORDS)
stopwords.update([" "]) #You can add stopwords if you have any 
wordcloud = WordCloud(stopwords=stopwords, background_color="white", width=800, height=400).generate(str(text))
plt.figure(figsize = (15, 20), facecolor = None) 
plt.imshow(wordcloud)
plt.axis("off")
plt.show()